In [5]:
import argparse
import nltk.data
import os
import json
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
#from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict
import re
import unicodedata
import sys
# from xml.etree import cElementTree as ET
import lxml.etree
from io import StringIO

from tqdm import tqdm
import nltk
nltk.download('punkt')

import numpy as np

[nltk_data] Downloading package punkt to /Users/ben/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
train_prop = 0.7
np.random.seed(0)

all_genders = open(r"gender_file.attr", 'r').readlines()
all_sent = open(r"text.txt", 'r').readlines()

male_idx = [id for id, gender in enumerate(all_genders) if gender.strip() == 'male']
female_idx = [id for id, gender in enumerate(all_genders) if gender.strip() == 'female']

num_male = len(male_idx)
male_idx_permuted = np.random.permutation(num_male)
train_num_male = int(train_prop * num_male)

num_female = len(female_idx)
female_idx_permuted = np.random.permutation(num_female)
train_num_female = int(train_prop * num_female)

training_idx, test_idx = list(male_idx_permuted[:train_num_male]) + list(female_idx_permuted[:train_num_female]), list(male_idx_permuted[train_num_male:]) + list(female_idx_permuted[train_num_female:])

In [7]:
GENDER_DICT = {"female":0, "male":1}
gender_doms = len(GENDER_DICT.keys())

train_gender_src_files = []
dev_gender_src_files = []
train_gender_trg_files = []
dev_gender_trg_files = []

for i in range(gender_doms):
    train_gender_src_files.append(open('blogs224n.train.src.gender.{}'.format(i), 'w+'))
    dev_gender_src_files.append(open('blogs224n.dev.src.gender.{}'.format(i), 'w+'))

    train_gender_trg_files.append(open('blogs224n.train.trg.gender.{}'.format(i), 'w+'))
    dev_gender_trg_files.append(open('blogs224n.dev.trg.gender.{}'.format(i), 'w+'))

train_gender_src_consolidated = open('blogs224n.train.src.txt', 'w+')
dev_gender_src_consolidated = open('blogs224n.dev.src.txt', 'w+')

train_gender_trg_consolidated = open('blogs224n.train.attr', 'w+')
dev_gender_trg_consolidated = open('blogs224n.dev.attr', 'w+')

for id in tqdm(training_idx):
    sent = all_sent[id]
    gender = all_genders[id].strip()
    gender_id = 0 if gender=='male' else 1

    train_gender_src_files[gender_id].write(sent)
    train_gender_trg_files[gender_id].write('dom'+str(gender_id)+'\n')

    train_gender_src_consolidated.write(sent)
    train_gender_trg_consolidated.write('dom'+str(gender_id)+'\n')

for id in tqdm(test_idx):
    sent = all_sent[id]
    gender = all_genders[id].strip()
    gender_id = 0 if gender=='male' else 1

    dev_gender_src_files[gender_id].write(sent)
    dev_gender_trg_files[gender_id].write('dom'+str(gender_id)+'\n')

    dev_gender_src_consolidated.write(sent)
    dev_gender_trg_consolidated.write('dom'+str(gender_id)+'\n')

100%|███████████████████████████████| 197196/197196 [00:00<00:00, 422886.04it/s]
